# **LAB SESSION 2 - Bagging and Random Forests**

## Utils

*Note: in all this project, use **the value 0** when you need to choose a value for a random state*

## Ex 1: comparison between random forests and bagging

We work with the [Urban Land Cover data base](https://archive.ics.uci.edu/ml/datasets/Urban+Land+Cover). The data are used for automated mapping of urban land cover (trees, grass, soil, concrete, asphalt, buildings, etc.) in satellite or aerial imagery. Nine types of urban land cover are considered and multi-scale spectral, size, shape, and texture information are used for classification. The data consists in a train set and a test set. **The goal is to predict the urban land cover (the variable named `class`) based on the multi-scale spectral, size, shape, and texture information. It is then a classification problem. We will use the overall accuracy (1-misclassification rate) as performance criterion**. Note that other preformnce criterion exist for classification problem such as specificity, sensitivity, F-score, etc.

Before to start, we: 
 1) load the data and perform a briel descriptive analysis of them;
 2) select the variables that we will use in the exercise, 


In [10]:
# Load the data 
import pandas as pd # data analysis
ulc_train = pd.read_csv("ULC_training.csv") 
ulc_test = pd.read_csv("ULC_testing.csv") 

In [ ]:
# Display the dimension
print(ulc_train.shape)
print(ulc_test.shape)

In [ ]:
# Have a first quick look at the datasets and display the dimension
ulc_train.head()

In [ ]:
ulc_test.head()

In [ ]:
## Display a statistic summary for the numerical variables
ulc_train.describe(include='all')

In [ ]:
ulc_test.describe(include='all')

In [ ]:
## Display the distribution of the target variable in the two datasets
ulc_train['class'].value_counts()


In [ ]:
ulc_test['class'].value_counts()

In [ ]:
# Select the variables that we be used 
target_column = "class" # The response variable that we will consider
features_columns = list(ulc_train)
features_columns.remove('class') # The predictors/features used to predict the target
#print(target_column)
#print(features_columns)

In [ ]:
data, target = ulc_train[features_columns], ulc_train[target_column]
data_test, target_test = ulc_test[features_columns], ulc_test[target_column]

Question 1. Fit a random forest named *`rfc`* on the train set to explain the type of urban land cover (variable `class`) according to multi-scale spectral, size, shape, and texture information. More specifically, you will use the [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) class. Read carefully the documentation.

*Indication: for the hyperparameters, use the values `n_estimators = 500` and `max_features= sqrt(d)` with `d` denoting the number of features, `oob_score=True`  and `random_state=0`.*




Before buiding `rfc` remind in the window below the meaning of`n_estimators` and `max_features`. 

Your answer:.............

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: create the object rfc, it is a RandomForestClassifier object with n_estimators=500, max_features='auto' and random_state=0
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(......)

# Step 2: build the random forest on the train set by indicating the input data and the target variable 
rfc.fit(.....)

In [ ]:
## Solution

# Step 1: create the object rfc, it is a RandomForestClassifier object with n_estimators=500, max_features='auto' and random_state=0
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=500, max_features='sqrt',oob_score=True,random_state=0)

# Step 2: build the random forest on the train set by indicating the input data and the target variable 
rfc.fit(data, target)

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 3: look at the parameters used by your forest
from pprint import pprint
print('Parameters of the forest:\n')
pprint(rfc.get_params())
print('\n')

Compute the oob error and comment the result. What does it represent ?

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##
# Step 4: print the oob_score (attributes of rfc named oob_score_)
.....

In [ ]:
# Step 4: print the oob_score (attributes of rfc named oob_score_)
print('OOB error:'),
print(rfc.oob_score_)

Meaning of the OOB error and result interpretation:

Comment: .............

Question 2. Predict the class of each observation of the test sample by using the random forest `rfc`and display the confusion matrix. Comment it. How many observations are misclassified ? Compute the accuracy.

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: compute the predictions on the test set
predictions_test = rfc.predict(.......) 

# Step 2: display the confusion matrix on the test set

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(....., .......)
disp = ConfusionMatrixDisplay(confusion_matrix=.....,display_labels=rfc.classes_)
disp.plot() 

# Step 3: compute the accuracy on the test set
accuracy = rfc.score(....,.....)
print(accuracy)


In [ ]:
## Solution

# Step 1: compute the predictions
predictions_test = rfc.predict(data_test) 

# Step 2: display the confusion matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(target_test, predictions_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=rfc.classes_)
disp.plot() 

# Step 3: compute the accuracy
accuracy = rfc.score(data_test,target_test)
print(accuracy)


In [ ]:
## Solution

# Step 1: compute the predictions
predictions_test = rfc.predict(data_test) 

# Step 2: display the confusion matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(target_test, predictions_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=rfc.classes_)
disp.plot() 

# Step 3: compute the accuracy
accuracy = rfc.score(data_test,target_test)
print(accuracy)


Comment: .........

Number of misclassified observations in the test set:............

Question 3. We will now focus on the calibration of some RF parameters: `n_estimators` and `max_features`. To calibrate these parameters, we will used the OOB errors. The code below shows how the OOB error can be measured at the addition of each new tree during training. The resulting plot can be used to approximate a suitable value of `n_estimators` at which the OOB error stabilizes. Comment this plot. What value for `n_estimators` does it seem suitable ?

*Indication: a suitable value for `n_estimators` is a value for which the oob error of the forest is stable.*


In [ ]:
import matplotlib.pyplot as plt

RANDOM_STATE = 0


# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = []

# Range of `n_estimators` values to explore.
min_estimators = 100
max_estimators = 800
step=5

for i in range(min_estimators, max_estimators+1, step):
    rf = RandomForestClassifier(warm_start=True, n_estimators=i, max_features='auto',random_state=RANDOM_STATE, oob_score=True)
    rf.fit(data, target)

    # Record the OOB error for each `n_estimators=i` setting.
    oob_error = 1 - rf.oob_score_
    error_rate.append(oob_error)
  

# Plot Generate the "OOB error rate" vs. "n_estimators" plot

plt.plot(range(min_estimators, max_estimators +1, step), error_rate, label="OOB error rate")

plt.ylim(0, 1.5*max(error_rate)) 
plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate vs. number of trees")
plt.legend(loc="upper right")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

RANDOM_STATE = 0 # Note: if you modifiy the random stat, the results can be sligthly different


# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = []

# Range of `n_estimators` values to explore.
min_estimators = 100
max_estimators = 800
step=5

for i in range(min_estimators, max_estimators+1, step):
    rf = RandomForestClassifier(warm_start=True, n_estimators=i, max_features='auto',random_state=RANDOM_STATE, oob_score=True)
    rf.fit(data, target)

    # Record the OOB error for each `n_estimators=i` setting.
    oob_error = 1 - rf.oob_score_
    error_rate.append(oob_error)
  

# Plot Generate the "OOB error rate" vs. "n_estimators" plot

plt.plot(range(min_estimators, max_estimators +1, step), error_rate, label="OOB error rate")

plt.ylim(0, 1.5*max(error_rate)) 
plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate vs. number of trees")
plt.legend(loc="upper right")
plt.show()

Comment of the plot and choice of a suitable value for `n_estimators`: .............






Question 4. Now, we will repeat `B` times a `k`-fold cross validation with the function GridSearchCV to calibrate at the same time the parameters `n_estimators` and `max_features`. Complete the command below, comment the results and select suitable values for the parameters `n_estimators` and `max_features`.

*Indications:*
- *the values considers for `n_estimators` and `max_features` are `max_features = (0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)` and `n_estimators=(400,600,800)`. The values (0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9) for `max_features` represent the proportions of selected features. See the documention of [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) class.*
- *we will repeat `B=10`times a k-fold crossvalidation with `k=3`.*

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: create a grid with all the values that we will considers for the two paramters
grid = {
    'max_features':[..........], 
    'n_estimators':[...........]
}

# Step 2: use the grid to to search for the best couple of parameters
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier() # create the forest model to tune

B=.......
results_cv=pd.DataFrame()
for i in range(B):
    
    rf_cv = GridSearchCV(estimator=rf,param_grid=grid, cv=......,n_jobs=-1)# Search the best values for the parameters using 3-fold cross validation, and use all available cores(n_jobs=-1)
    rf_cv.fit(data, target) # Fit the CV search
    if i==0: 
        results_cv=pd.DataFrame(rf_cv.cv_results_)[["params","mean_test_score","std_test_score"]]
    else:
        results_cv["mean_test_score"]=results_cv["mean_test_score"]+pd.DataFrame(rf_cv.cv_results_)["mean_test_score"]
        results_cv["std_test_score"]=results_cv["std_test_score"]+pd.DataFrame(rf_cv.cv_results_)["std_test_score"]  
        
        

results_cv["mean_test_score"]=results_cv["mean_test_score"]/B
results_cv["std_test_score"]=results_cv["std_test_score"]/B 


# Step 3: get the best parameters (with the higher performance)
ind_best=results_cv["mean_test_score"].idxmax()
print(results_cv["params"].iloc[ind_best])
print(results_cv["mean_test_score"].iloc[ind_best])
print(results_cv["std_test_score"].iloc[ind_best])
print(results_cv)

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: create a grid with all the values that we will considers for the two paramters
grid = {
    'max_features':[..........], 
    'n_estimators':[...........]
}

# Step 2: use the grid to to search for the best couple of parameters
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier() # create the forest model to tune

B=.......
results_cv=pd.DataFrame()
for i in range(B):
    
    rf_cv = GridSearchCV(estimator=rf,param_grid=grid, cv=......,n_jobs=-1)# Search the best values for the parameters using 3-fold cross validation, and use all available cores(n_jobs=-1)
    rf_cv.fit(data, target) # Fit the CV search
    if i==0: 
        results_cv=pd.DataFrame(rf_cv.cv_results_)[["params","mean_test_score","std_test_score"]]
    else:
        results_cv["mean_test_score"]=results_cv["mean_test_score"]+pd.DataFrame(rf_cv.cv_results_)["mean_test_score"]
        results_cv["std_test_score"]=results_cv["std_test_score"]+pd.DataFrame(rf_cv.cv_results_)["std_test_score"]  
        
        

results_cv["mean_test_score"]=results_cv["mean_test_score"]/B
results_cv["std_test_score"]=results_cv["std_test_score"]/B 


# Step 3: get the best parameters (with the higher performance)
ind_best=results_cv["mean_test_score"].idxmax()
print(results_cv["params"].iloc[ind_best])
print(results_cv["mean_test_score"].iloc[ind_best])
print(results_cv["std_test_score"].iloc[ind_best])
print(results_cv)

Result interpretation and choice of tuned values for `n_estimators` and `max_features`: ........

Solution for the interpretation and the selection of values for `n_estimators` and `max_features`: .......

Question 5. Build the random forest `opt_rfc` by using the best values for the parameters `n_estimators` and `max_features`. Compute the accuracy on the test set and display the confusion matrix. 

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: fit a random forest with the best values for parameters
opt_rfc = RandomForestClassifier(n_estimators=...., max_features=....,oob_score=True,random_state=0)

# Step 2: build the random forest on the train set by indicating the input data and the target variable 
opt_rfc.fit(...,.....)

# Step 3: compute the accuracy and the confusion matrix
predictions_test_2 = opt_rfc.predict(....) 
cm_2 = confusion_matrix(...., .....)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_2,display_labels=opt_rfc.classes_)
disp.plot() 

# Step 4: compute the accuracy
accuracy_2 = opt_rfc.score(....,.....)
print(accuracy_2)

Question 6. what value for`max_features` we have to choose if we want to apply the bagging algorithm with CART instead of a random forest ? Build this model and compute the prediction error of this model based on the test set. We will call this model $bag$.

*Indication: use the value selected at `question 5`for the parameter `n_estimators`.* 

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: fit a a bagging model (use the best value for n_estimators)
bag = RandomForestClassifier(n_estimators=...., max_features=....,oob_score=True,random_state=0)

# Step 2: build the random forest on the train set by indicating the input data and the target variable 
bag.fit(...., ......)

# Step 3: compute the prediction on the test set and the confusion matrix
predictions_test_bag = bag.predict(......) 
cm_bag = confusion_matrix(.....,......)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_bag,display_labels=bag.classes_)
disp.plot() 

# Step 4: compute the accuracy on the test set
accuracy_bag = bag.score(....,.....)
print(accuracy_bag)



Question 8. Compare your three models `opt_rfc`, `bag`and `rfc` using suitable performance criteria. What model do you choose and why ?

Your Answer: ........




Question 9. The code below shows the boxplot of the mean decrease in accuracy (MDA) for the 15 features with the highest average MDA. The MDA of each of the 147 features has been independently computed `n__repeats =10` times on the test set for the random forest `opt_rfc` (at each run, another permutation is applied for the features). Comment the plot. 

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(opt_rfc, data_test, target_test, n_repeats=10, random_state=0, n_jobs=2)

sorted_idx = result.importances_mean.argsort()
invert_sorted_idx=sorted_idx[::-1][:14]# keep only the 15 features with the highest average MDA

fig, ax = plt.subplots()
ax.boxplot(result.importances[invert_sorted_idx].T,
           vert=False, labels=data_test.columns[invert_sorted_idx])
ax.set_title("MDA computed on the test set for the 15 features with the highest average MDA (average MDA is displayed in orange")
fig.tight_layout()
plt.show()

Comments about the plot above: .......

## Ex 2: introduction to regression trees

Here, we consider the dataset named *srbct_data*. It is relative to small round blue cell tumors of childhood. This set is composed of :

- a response factor of length 63, called class, indicating the class of each sample (4 classes in total).
- 2308 predictors. Each predictor represents the expression of one gene. The features are correlated. 

More information about the data are available on https://www.rdocumentation.org/packages/plsgenomics/versions/1.5-2/topics/SRBCT 

The table named *genes_name* contains the names of the genes and a description for each gene.

In [ ]:
# Load the data 
import pandas as pd # data analysis
srbct_data = pd.read_csv("cancer_data.csv")
genes_name = pd.read_csv("cancer_data_genes_names.csv")

In [ ]:
# Have a look at the data 
srbct_data.head()

In [ ]:
genes_name.head()

In [ ]:
# Display the dimension and a brief statistics summary
srbct_data.shape

In [ ]:
# Display a brief statistics summary
srbct_data.describe(include='all')

In [ ]:
## Display a frequency table for the target variable named `class`.
srbct_data["class"].value_counts()

In [ ]:
# Select the variables that we be used 
target_name = "class" # The response variable that we will consider
features_names = list(srbct_data)
features_names.remove('class') # The predictors/features used to predict the target
#print(target_column)
#print(features_columns)

In [ ]:
# Split the dataset into train (75% of data) and test dataset (25% of data)
from sklearn.model_selection import train_test_split
X, y =  srbct_data[features_names], srbct_data[target_name]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,test_size=0.25)
print(X_train.shape)
print(X_test.shape)

**Questions:**
 - 1) Build a random forest on this dataset by using the [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor) class. Read carefully the documentation and use the default values for the RF parameters.
 - 2) Calibrate the two parameters `n_estimators` and `max_features` by using the same approach as in `Ex1 question 4`. Then, select values for `n_estimators` and `max_features`.
 - 3) Build a second random forest using the selected values for `n_estimators` and `max_features`.
 - 4) Because there are lots of features and they are correlated, use the MDA score to select a subset of only 20 variables (use the same approach as in `Ex1 question 9`). Justify your choice.
 - 5) Build a third random forest based only the selected subset of features.
 - 6) Compare the three models. What model do you select and why ?

In [ ]:
##---- Write your answer ----##






